In [1]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [2]:
import datajoint as dj
from datetime import datetime
from workflow.pipeline import culture, ephys

[2024-01-24 21:35:54,459][INFO]: Connecting milagros@db.datajoint.com:3306
[2024-01-24 21:35:55,812][INFO]: Connected milagros@db.datajoint.com:3306


The objective of this notebook is to guide you through the process of inserting an ephys session with the `session_type` parameter set to `spike_sorting`. If there is already an entry in the table for this session type or others (such as `lfp`), you may consider updating the `session_type` to include both LFP and spike sorting analyses, as explained later in this notebook.

## Manually Add an Ephys Session for Spike Sorting

### Choose the organoid and experiment

The `Experiment` table contains the experiment to be performed on each organoid under different drug conditions. 

In [3]:
culture.Experiment()

organoid_id e.g. O17,experiment_start_time,experiment_end_time,user,lineage_id de-identified code (e.g. hmau001),induction_culture_date,induction_culture_plate,post_induction_culture_date,post_induction_culture_plate,isolated_rosette_culture_date Date for isolating the rosette,isolated_rosette_culture_plate,organoid_culture_date,organoid_culture_plate,drug_name,drug_concentration concentration in uM,"experiment_plan e.g. mrna lysate, oct, protein lysate, or matrigel embedding, ephys, tracing"
O09,2023-05-18 12:25:00,2023-05-18 18:15:00,None,None,None,None,None,None,None,None,None,None,4-AP,100.0,ephys
O09,2023-05-18 18:15:00,2023-05-19 09:30:00,None,None,None,None,None,None,None,None,None,None,No Drug,nan,ephys
O09,2023-05-19 09:30:00,2023-05-19 15:35:00,None,None,None,None,None,None,None,None,None,None,Bicuculline,50.0,ephys
O09,2023-05-19 15:45:00,2023-05-20 15:40:00,None,None,None,None,None,None,None,None,None,None,Tetrodotoxin,1.0,ephys
O10,2023-05-18 12:25:00,2023-05-18 18:15:00,None,None,None,None,None,None,None,None,None,None,4-AP,100.0,ephys
O10,2023-05-18 18:15:00,2023-05-19 09:30:00,None,None,None,None,None,None,None,None,None,None,No Drug,nan,ephys
O10,2023-05-19 09:30:00,2023-05-19 15:35:00,None,None,None,None,None,None,None,None,None,None,Bicuculline,50.0,ephys
O10,2023-05-19 15:45:00,2023-05-20 15:40:00,None,None,None,None,None,None,None,None,None,None,Tetrodotoxin,1.0,ephys
O11,2023-05-18 12:25:00,2023-05-18 18:15:00,None,None,None,None,None,None,None,None,None,None,4-AP,100.0,ephys
O11,2023-05-18 18:15:00,2023-05-19 09:30:00,None,None,None,None,None,None,None,None,None,None,No Drug,nan,ephys


Let's choose one specific experiment for a organoid ID and a experiment start time:

In [4]:
culture.Experiment & "organoid_id='O13'" & "experiment_start_time='2023-06-08 19:05:00'"

organoid_id e.g. O17,experiment_start_time,experiment_end_time,user,lineage_id de-identified code (e.g. hmau001),induction_culture_date,induction_culture_plate,post_induction_culture_date,post_induction_culture_plate,isolated_rosette_culture_date Date for isolating the rosette,isolated_rosette_culture_plate,organoid_culture_date,organoid_culture_plate,drug_name,drug_concentration concentration in uM,"experiment_plan e.g. mrna lysate, oct, protein lysate, or matrigel embedding, ephys, tracing"
O13,2023-06-08 19:05:00,2023-06-08 23:35:00,None,None,None,None,None,None,None,None,None,None,4-AP,100.0,ephys


Let's define a single session for the chosen experiment with the `session_info` and `session_probe_info` keys using the experiment information: 
- the organoid ID.
- the start and end time.
- the `probe`, `port_id`, and `used_electrodes`. 

The `data/ephys_session.yml` contains some examples from previous experiments. Please, use the information from your current experiment here.  

In [17]:
# Modify this cell to insert sessions
session_info = dict(
    organoid_id="O13",
    insertion_number=0,
    experiment_start_time="2023-06-08 19:05:00",
    start_time="2023-06-08 19:10:00",
    end_time="2023-06-09 19:15:00",
    session_type="spike_sorting",
)

session_probe_info = dict(
    organoid_id="O13",
    insertion_number=0,
    experiment_start_time="2023-06-08 19:05:00",
    start_time="2023-06-08 19:10:00",
    end_time="2023-06-09 19:15:00",
    probe="Q983",  # probe serial number
    port_id="A",  # Port ID ("A", "B", etc.)
    used_electrodes=[
        0,
        1,
        2,
        3,
        4,
        5,
        6,
        7,
        8,
        9,
        10,
        11,
        12,
        13,
        14,
        15,
        16,
        17,
        18,
        19,
        20,
        21,
    ],  # electrodes used for the session
)

Verify that the key is correct before inserting it:

In [6]:
session_info

{'organoid_id': 'O13',
 'insertion_number': 0,
 'experiment_start_time': '2023-06-08 19:05:00',
 'start_time': '2023-06-08 19:10:00',
 'end_time': '2023-06-09 19:15:00',
 'session_type': 'spike_sorting'}

In [7]:
session_probe_info

{'organoid_id': 'O13',
 'insertion_number': 0,
 'experiment_start_time': '2023-06-08 19:05:00',
 'start_time': '2023-06-08 19:10:00',
 'end_time': '2023-06-09 19:15:00',
 'probe': 'Q983',
 'port_id': 'A',
 'used_electrodes': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21]}

### Insert the session in the `EphysSession` table

With the next step, you'll add a new session for this experiment, and the worker will then ingest it.

In [8]:
# Insert the session
ephys.EphysSession.insert1(session_info, ignore_extra_fields=True)
ephys.EphysSessionProbe.insert1(session_probe_info, ignore_extra_fields=True)

DuplicateError: ("Duplicate entry 'O13-2023-06-08 19:05:00-0-2023-06-08 19:10:00-2023-06-09 19:15:0' for key 'ephys_session.PRIMARY'", 'To ignore duplicate entries in insert, set skip_duplicates=True')

In [9]:
(culture.Experiment & session_info)

organoid_id e.g. O17,experiment_start_time,experiment_end_time,user,lineage_id de-identified code (e.g. hmau001),induction_culture_date,induction_culture_plate,post_induction_culture_date,post_induction_culture_plate,isolated_rosette_culture_date Date for isolating the rosette,isolated_rosette_culture_plate,organoid_culture_date,organoid_culture_plate,drug_name,drug_concentration concentration in uM,"experiment_plan e.g. mrna lysate, oct, protein lysate, or matrigel embedding, ephys, tracing"
O13,2023-06-08 19:05:00,2023-06-08 23:35:00,None,None,None,None,None,None,None,None,None,None,4-AP,100.0,ephys


### To verify the occurrence of this Error: `DuplicateError`

If a `DuplicateError` appeared, it means that this session has been already inserted in the table. It is likely that this session was inserted for a `session_type = 'lfp'`. You can check this with the following steps:

In [ ]:
# session_info["session_type"]='lfp'
# (ephys.EphysSession & session_info)

Do not forget to update again the `session_type` in the `session_info` key before inserting it again in the tables.

### To update the `session_type` from 'lfp' to 'both'

If you want this session to be analyzed both for LFP and spike sorting, and the session is already inserted with `session_type = 'lfp'`, you might want to update this session type to `both` (LFP and spike sorting):

In [ ]:
# session_info['session_type']='both'

In [ ]:
# ephys.EphysSession.update1(session_info)

In [ ]:
# ephys.EphysSession & session_info

`EphysSessionProbe` does not need to be updated since this table does not have `session_type` secondary attribute.

### To review all the ephys sessions inserted for `spike_sorting` and `both`

In [11]:
(ephys.EphysSession & "session_type = 'spike_sorting'")

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,session_type
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,spike_sorting
O13,2023-06-08 19:05:00,0,2023-06-08 19:05:00,2023-06-09 19:15:00,spike_sorting
O13,2023-06-08 19:05:00,0,2023-06-08 19:05:00,2023-06-09 23:35:00,spike_sorting


In [12]:
(ephys.EphysSession & "session_type = 'both'")

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,session_type
O13,2023-06-08 19:05:00,0,2023-06-08 19:10:00,2023-06-09 19:15:00,both


### 

### To delete entries in the tables, use the `.delete()` carefully

In [ ]:
# Example
# (ephys.EphysSession & "session_type = 'both'").delete()